# Génération de partition de musique : réseau LSTM

**/!\ Les notes et leurs durées sont séparées lors de la phase d'apprentissage**

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from typing import Dict, List, Optional, Sequence, Tuple
import numpy as np
import pandas as pd
import music21
import pretty_midi

import json
import os
import sys
sys.path.append('C:/Users/melan/Documents/M2_S10 IAFA/CHEF D\'OEUVRE')
import extract_data as ed
import visualization as vz

c:\users\melan\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


### Choix de la partie à générer

In [2]:
part = 'A'

### Préparation du dataset

In [3]:
# Creating the required variables
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
with open("data/data.json",'r') as file:
        data = json.load(file)
                
filenames = [data[i]['title'] for i in range(len(data))]
print('Number of files:', len(filenames))

Number of files: 171


In [5]:
def extract_notes_from(jsonfile,score,part):
   
    title,part,key,notes_duration = ed.extract_seq_from(jsonfile,score,part)
    
    notes = {'name':[],'duration':[]}
    
    for nd in notes_duration:
        
        splitted = nd.split('-')
        if len(splitted) == 2:
            if splitted[0] == 'rest':
                only_note,duration = 128,float(splitted[1])
            else:                
                only_note, duration = music21.note.Note(splitted[0]).pitch.midi, float(splitted[1])
        else:
            only_note = music21.note.Note("b".join(substring for substring in splitted[:-1])).pitch.midi
            duration = float(splitted[-1])
        notes['name'].append(only_note)
        notes['duration'].append(duration)

    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

sample_file = filenames[0]
print(sample_file)

raw_notes = extract_notes_from("data/data.json",'A Ginga do Man\u00e9',part)
raw_notes.head()

A Ginga do Mané


,name,duration
0,88,0.25
1,79,0.25
2,80,0.25
3,81,0.25
4,82,0.25


### Création du dataset d'entraînement

In [6]:
num_files = len(filenames)

all_notes = []

for f in filenames:
    
    notes = extract_notes_from("data/data.json",f,part)
    all_notes.append(notes)

all_notes = pd.concat(all_notes)

n_notes = len(all_notes)
print('Number of notes parsed for part',part,':', n_notes)

key_order = ['name', 'duration']
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)

notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)

notes_ds.element_spec

Number of notes parsed for part A : 19175


TensorSpec(shape=(2,), dtype=tf.float64, name=None)

In [7]:
def create_sequences(dataset, seq_length):
    """Returns TF Dataset of sequence and label examples."""
    
    seq_length = seq_length+1

    # Take 1 extra for the labels
    windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)

    # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
    flatten = lambda x: x.batch(seq_length, drop_remainder=True)
    sequences = windows.flat_map(flatten)

    # Split the labels
    def split_labels(sequences):
        inputs = sequences[:-1]
        labels_dense = sequences[-1]
        labels = {key:labels_dense[i] for i,key in enumerate(key_order)}

        return inputs,labels

    return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)
    
seq_length = 50
seq_ds = create_sequences(notes_ds, seq_length)

In [8]:
batch_size = 64
buffer_size = n_notes - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

### Développement du modèle

In [9]:
def mse_with_positive_pressure(y_true, y_pred):
    mse = (y_true - y_pred) ** 2
    positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
    return tf.reduce_mean(mse + positive_pressure)

In [10]:
# Developing the model

input_shape = (seq_length, 2)
learning_rate = 0.005

inputs = Input(input_shape)
x = LSTM(129)(inputs)

outputs = {'name': Dense(129, name='name')(x),
           'duration': Dense(1, name='duration')(x),
          }

model = Model(inputs, outputs)

loss = {'name': SparseCategoricalCrossentropy(from_logits=True),
        'duration': mse_with_positive_pressure,
       }

optimizer = Adam(learning_rate=learning_rate)

model.compile(loss=loss, optimizer=optimizer)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 2)]              0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 129)                  68112     ['input_1[0][0]']             
                                                                                                  
 duration (Dense)            (None, 1)                    130       ['lstm[0][0]']                
                                                                                                  
 name (Dense)                (None, 129)                  16770     ['lstm[0][0]']                
                                                                                             

In [11]:
# Creating the necessary callbacks

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./training_checkpoints/ckpt_{epoch}', save_weights_only=True),
             tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, 
                                              verbose=1, restore_best_weights=True),]

In [12]:
# Compiling and fitting the model

model.compile(loss = loss, 
              loss_weights = {'name': 0.05, 'duration':1.0,},
              optimizer = optimizer)

epochs = 50

history = model.fit(train_ds, 
                    epochs=epochs, 
                    callbacks=callbacks,)

Epoch 1/50


298/298 [==============================] - 17s 43ms/step - loss: 0.2290 - duration_loss: 0.0725 - name_loss: 3.1294
Epoch 2/50
298/298 [==============================] - 12s 39ms/step - loss: 0.2060 - duration_loss: 0.0595 - name_loss: 2.9309
Epoch 3/50
298/298 [==============================] - 12s 40ms/step - loss: 0.2012 - duration_loss: 0.0586 - name_loss: 2.8535
Epoch 4/50
298/298 [==============================] - 12s 41ms/step - loss: 0.1975 - duration_loss: 0.0576 - name_loss: 2.7973
Epoch 5/50
298/298 [==============================] - 12s 40ms/step - loss: 0.1946 - duration_loss: 0.0572 - name_loss: 2.7487
Epoch 6/50
298/298 [==============================] - 12s 40ms/step - loss: 0.1927 - duration_loss: 0.0567 - name_loss: 2.7197
Epoch 7/50
298/298 [==============================] - 12s 40ms/step - loss: 0.1913 - duration_loss: 0.0563 - name_loss: 2.7007
Epoch 8/50
298/298 [==============================] - 12s 41ms/step - loss: 0.1897 - duration_loss: 0.0556 - 

### Génération de notes

In [13]:
def converting_value_to_note(value):
    if value == 128:
        return "rest"
    else:
        n = music21.pitch.Pitch()
        n.midi = round(value)
        return str(n.nameWithOctave)

In [14]:
def predict_next_note(notes, model, temperature = 1.0):
    """Generates a note IDs using a trained sequence model."""

    assert temperature > 0

    # Add batch dimension
    inputs = tf.expand_dims(notes, 0)

    predictions = model.predict(inputs)
    name_logits = predictions['name']
    duration = predictions['duration']

    name_logits /= temperature
    name = tf.random.categorical(name_logits, num_samples=1)
    name = tf.squeeze(name, axis=-1)
    duration = tf.squeeze(duration, axis=-1)

    quarter_duration = [0.0625,0.125,0.25,0.5,1.0,2.0]

    duration = min(quarter_duration, key=lambda x:abs(x-duration))

    return int(name), float(duration)

In [15]:
def generate_x_notes(x,initial_notes,input_notes,temperature):
    
    generated_notes = []
    
    for _ in range(x):
        
        name, duration = predict_next_note(input_notes, model, temperature)
        converted_name = converting_value_to_note(name)

        # si ce n'est pas la première générée
        if generated_notes != []:
            
            splitted = generated_notes[-1].split("-")

            # et si elle est identique à la précédente
            if splitted[0] == converted_name:
                new_duration = float(splitted[1]) + duration
                new_note = "-".join([splitted[0],str(new_duration)])
                generated_notes[-1] = new_note
                
                #print(new_note)

            # si elle n'est pas identique à la précédente
            else:
                new_note = [converted_name,str(duration)]
                new_note = "-".join(substring for substring in new_note)

                generated_notes.append(new_note)

        # si c'est la première note générée
        else:
            new_note = [converted_name,str(duration)]
            new_note = "-".join(substring for substring in new_note)

            generated_notes.append(new_note)
            
        
        input_note = (name, duration)
        input_notes = np.delete(input_notes, 0, axis=0)
        input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)
        
    return generated_notes

In [24]:
def alternatively_original_generated(jsonfile,score,part,temperature=2.0,x=8):
    
    raw_notes = extract_notes_from(jsonfile,score,part)
    sample_notes = np.stack([raw_notes[key] for key in key_order], axis=1)
    
    original_notes = [[pretty_midi.note_number_to_name(name),str(duration)] for name,duration in sample_notes]
    original_notes = ["-".join(substring for substring in note) for note in original_notes]
    
    length = sample_notes.shape[0]
    
    generated_sequence = []
    
    for i in range(0,length,x*2):
                
        input_notes = sample_notes[i:i+x]
        initial_notes = original_notes[i:i+x]
        generated_notes = generate_x_notes(x,initial_notes,input_notes,temperature)
        generated_sequence += initial_notes + generated_notes
        
    return {'Title':score,'Part':part,'Key':"F major",'Start_sequence':initial_notes,'Generated':generated_sequence,'Original':original_notes}

In [27]:
generated_scores = []

for f in filenames[:10]:
    
    print("ICI il y a I :",f)
    gen_notes = alternatively_original_generated("data/data.json",f,part,x=8)
    generated_scores.append(gen_notes)
    
with open('Generated/LSTM_'+part+'2.json','w') as file:
    json.dump(generated_scores,file) 

ICI il y a I : A Ginga do Mané
1/1 [==============================] - 0s 31ms/step
ICI il y a I : A Vida é um Buraco
1/1 [==============================] - 0s 31ms/step
ICI il y a I : ACARICIANDO
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 31ms/step
ICI il y a I : ACERTA O PASSO
1/1 [==============================] - 0s 13ms/step
ICI il y a I : Aeroporto do Galeao
1/1 [==============================] - 0s 32ms/step
ICI il y a I : AGRADECENDO
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step
ICI il y a I : Aguenta Seu Fulgêncio
1/1 [==============================] - 0s 26ms/step
ICI il y a I : Ainda me Recordo
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 31ms/step
ICI il y a I : AMAPA
1/1 [==============================] - 0s 31ms/step
ICI il y a I : Ameno Resedá
1/1 [==============================] - 0s 31ms/step


In [28]:
music21.environment.set('musescoreDirectPNGPath',str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
music21.environment.set('musicxmlPath', str(os.path.join("C:\\", "Program Files","MuseScore 4","bin","MuseScore4.exe")))
vz.show_all_generated("Generated/LSTM_"+part+"2.json").show("musicxml")